#### Example of using crosswalks between census tracts of different years

This notebook used to provide example code for using the output from the script in creating geojsons with multiple years of census data for the same tract. As I'm migrating it to a dedicated script in main.py, it's now for testing different parts of that script.  

#### 1.) Download the crosswalks from the Azure container 

In [3]:
import utils

In [1]:
from tract_crosswalk import get_tract_crosswalks

get_tract_crosswalks()

2023-12-14 09:30:15,548 - logger - INFO - (OVERWRITE_AZURE=False, OVERWRITE_LOCAL=False)
2023-12-14 09:30:15,710 - logger - INFO - Checking if crosswalks exist:
{'file': 'convert-ctracts_pct-area_2010-to-2020.json', 'local': True, 'azure': True}
{'file': 'convert-ctracts_pct-area_2020-to-2010.json', 'local': True, 'azure': True}
2023-12-14 09:30:15,711 - logger - INFO - Local crosswalks exist and OVERWRITE_LOCAL=False. Aborting script.


In [4]:
map_10_to_20 = utils.read_json_rows('convert-ctracts_pct-area_2010-to-2020.json')



FileNotFoundError: [Errno 2] No such file or directory: 'convert-ctracts_pct-area_2010-to-2020.json'

In [4]:
from tract_crosswalk import load_crosswalk_2010_2020
import pandas as pd

map_10_to_20 = load_crosswalk_2010_2020()

In [5]:
df = pd.DataFrame(map_10_to_20)

In [6]:
df

,STATENAME,GEOID_TRACT_10,GEOID_TRACT_20_overlap
0,Alabama,01001020100,"{'01001020100': 0.999, '01001020803': 0.0}"
1,Alabama,01001020200,"{'01001020100': 0.0, '01001020200': 1.0}"
2,Alabama,01001020300,{'01001020300': 1.0}
3,Alabama,01001020400,{'01001020400': 1.0}
4,Alabama,01001020500,"{'01001020501': 1.0, '01001020502': 1.0, '0100..."
...,...,...,...
74353,Wyoming,56043000200,"{'56043000200': 1.0, '56043000302': 0.008}"
74354,Wyoming,56043000301,{'56043000301': 1.0}
74355,Wyoming,56043000302,"{'56043000200': 0.0, '56043000302': 0.992}"
74356,Wyoming,56045951100,"{'56045951100': 1.0, '56045951300': 0.0}"


In [1]:
import utils 

utils.construct_raw_census_output_path()

'data/raw/B01001A_012E-B01001A_017E-B19013_001E_allStates+DC_2010-2020.csv'

In [2]:
from census_data import download_raw_data

df = download_raw_data()

2023-12-14 11:19:44,260 - logger - INFO - Downloading ['B01001A_012E', 'B01001A_017E', 'B19013_001E'] from 2010 to 2020: data/raw/B01001A_012E-B01001A_017E-B19013_001E_allStates+DC_2010-2020.csv
2023-12-14 11:19:45,723 - logger - INFO - Downloaded (AL, 2010, 1s, 1181 rows, 0.5581798553466797MB)
2023-12-14 11:19:45,724 - logger - INFO - NAME, B01001A_012E, B01001A_017E, B19013_001E, state, county, tract, year, state_fips, state_name, state_usps

2023-12-14 11:19:46,988 - logger - INFO - Downloaded (AL, 2011, 1s, 1181 rows, 0.5581798553466797MB)
2023-12-14 11:19:46,989 - logger - INFO - NAME, B01001A_012E, B01001A_017E, B19013_001E, state, county, tract, year, state_fips, state_name, state_usps

2023-12-14 11:19:48,354 - logger - INFO - Downloaded (AL, 2012, 1s, 1181 rows, 0.5581798553466797MB)
2023-12-14 11:19:48,355 - logger - INFO - NAME, B01001A_012E, B01001A_017E, B19013_001E, state, county, tract, year, state_fips, state_name, state_usps

2023-12-14 11:19:49,773 - logger - INFO - D

In [1]:
import pandas as pd 
import utils
import census_data
import yaml
from tract_crosswalk import load_crosswalk_2010_2020

# census_data.download_raw_data()

with open('config.yaml', 'r') as file: 
    config = yaml.full_load(file)

raw_census_table_path = utils.construct_raw_census_output_path()
raw_census_table = pd.read_csv(raw_census_table_path, index_col=0, dtype={'state_fips':str})


import census_data
long_df = census_data._transform_raw_data_long(raw_census_table)
wide_df = census_data._widen_df(long_df)
df_2020 = census_data._extract_2020_data(wide_df)
collapsed_df = census_data._collapse_df(wide_df)

# Load crosswalk from 2010 ot 2020
df_map_10_to_20 = pd.DataFrame(load_crosswalk_2010_2020(reverse=False))

joined = census_data.join_crosswalk(collapsed_df)
# joined = joined.dropna()

weighted_df = census_data.apply_crosswalk(joined)

2023-12-14 18:12:37,456 - logger - DEBUG - Raw Census data in data/raw/B19013_001E-B01001A_017E_DE-MD-NJ-PA-RI+DC_2010-2020.csv:
NAME, B19013_001E, B01001A_017E, state, county, tract, year, state_usps
"Census Tract 169.04,  New Castle County,  Delaware", 70875.0, 668.0, 10, 3, 16904, 2010, DE
"Census Tract 9801,  New Castle County,  Delaware", -666666666.0, 0.0, 10, 3, 980100, 2010, DE
"Census Tract 9901,  New Castle County,  Delaware", -666666666.0, 0.0, 10, 3, 990100, 2010, DE
"Census Tract 501.01,  Sussex County,  Delaware", 52569.0, 1510.0, 10, 5, 50101, 2010, DE
(78574, 8)

2023-12-14 18:12:37,869 - logger - DEBUG - 934 rows have negative placeholder values for census variables:
 GEOID, state_fips, state_name, state_usps, county_fips, county_name, tract_fips, tract_dec, year, B19013_001E, B01001A_017E, count
10001980000, 10, Delaware, DE, 001, Kent, 980000, 9800, 2020, -666666666.0, 48.0, 1
42019910500, 42, Pennsylvania, PA, 019, Butler, 910500, 9105, 2012, -666666666.0, 696.0, 1


In [2]:
long_df.isna().sum()

GEOID           0
state_fips      0
state_name      0
state_usps      0
county_fips     0
county_name     0
tract_fips      0
tract_dec       0
year            0
B19013_001E     4
B01001A_017E    0
dtype: int64

In [5]:
long_df.value_counts('year')

year
2020    7614
2010    7096
2011    7096
2012    7096
2013    7096
2014    7096
2015    7096
2016    7096
2017    7096
2018    7096
2019    7096
Name: count, dtype: int64

In [11]:
raw_census_table[raw_census_table.isna().sum(axis=1) >0]

,NAME,B19013_001E,B01001A_017E,state,county,tract,year,state_usps
110,"Census Tract 9822, Allegheny County, Pennsylvania",NaN,2018.0,42,3,982200,2015,PA
313,"Census Tract 166, Philadelphia County, Pennsyl...",NaN,177.0,42,101,16600,2015,PA
184,"Census Tract 9804, Somerset County, Maryland",NaN,12.0,24,39,980400,2015,MD
294,"Census Tract 4906.05, Baltimore County, Maryland",NaN,1769.0,24,5,490605,2015,MD


In [7]:
long_df[long_df['year'] == 2020]

,GEOID,state_fips,state_name,state_usps,county_fips,county_name,tract_fips,tract_dec,year,B19013_001E,B01001A_017E
0,10003014200,10,Delaware,DE,003,New Castle,014200,142,2020,80063.0,754.0
1,10003014301,10,Delaware,DE,003,New Castle,014301,143.01,2020,121312.0,567.0
2,10003014302,10,Delaware,DE,003,New Castle,014302,143.02,2020,111579.0,2910.0
3,10003014402,10,Delaware,DE,003,New Castle,014402,144.02,2020,76705.0,1134.0
4,10003014403,10,Delaware,DE,003,New Castle,014403,144.03,2020,75625.0,1716.0
...,...,...,...,...,...,...,...,...,...,...,...
245,44007010501,44,Rhode Island,RI,007,Providence,010501,105.01,2020,69185.0,2246.0
246,44007010502,44,Rhode Island,RI,007,Providence,010502,105.02,2020,70612.0,1904.0
247,44007010600,44,Rhode Island,RI,007,Providence,010600,106,2020,58203.0,2758.0
248,44007010701,44,Rhode Island,RI,007,Providence,010701,107.01,2020,91938.0,968.0


In [4]:
wide_df.isna().sum()

B19013_001E-2010     1138
B19013_001E-2011     1138
B19013_001E-2012     1138
B19013_001E-2013     1138
B19013_001E-2014     1138
B19013_001E-2015     1142
B19013_001E-2016     1138
B19013_001E-2017     1138
B19013_001E-2018     1138
B19013_001E-2019     1138
B01001A_017E-2010    1138
B01001A_017E-2011    1138
B01001A_017E-2012    1138
B01001A_017E-2013    1138
B01001A_017E-2014    1138
B01001A_017E-2015    1138
B01001A_017E-2016    1138
B01001A_017E-2017    1138
B01001A_017E-2018    1138
B01001A_017E-2019    1138
dtype: int64

In [45]:
df_2020_raw = raw_census_table[raw_census_table['year'] == 2020]
df_2020_raw['state'] = df_2020_raw['state'].apply(lambda x: utils.std_fips(x, geog = 'state'))
df_2020_raw['county'] = df_2020_raw['county'].apply(lambda x: utils.std_fips(x, geog = 'county'))
df_2020_raw['tract'] = df_2020_raw['tract'].apply(lambda x: utils.std_fips(x, geog = 'tract'))

df_2020_raw['GEOID'] = df_2020_raw['state'] + df_2020_raw['county'] + df_2020_raw['tract']
df_2020_raw


/var/folders/qy/qp1k23kj0jj0bpkn10sx9mhh0000gn/T/ipykernel_21472/3393103179.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020_raw['state'] = df_2020_raw['state'].apply(lambda x: utils.std_fips(x, geog = 'state'))
/var/folders/qy/qp1k23kj0jj0bpkn10sx9mhh0000gn/T/ipykernel_21472/3393103179.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020_raw['county'] = df_2020_raw['county'].apply(lambda x: utils.std_fips(x, geog = 'county'))
/var/folders/qy/qp1k23kj0jj0bpkn10sx9mhh0000gn/T/ipykernel_2147

,NAME,B19013_001E,B01001A_017E,state,county,tract,year,state_usps,GEOID
0,"Census Tract 142, New Castle County, Delaware",80063.0,754.0,10,003,014200,2020,DE,10003014200
1,"Census Tract 143.01, New Castle County, Delaware",121312.0,567.0,10,003,014301,2020,DE,10003014301
2,"Census Tract 143.02, New Castle County, Delaware",111579.0,2910.0,10,003,014302,2020,DE,10003014302
3,"Census Tract 144.02, New Castle County, Delaware",76705.0,1134.0,10,003,014402,2020,DE,10003014402
4,"Census Tract 144.03, New Castle County, Delaware",75625.0,1716.0,10,003,014403,2020,DE,10003014403
...,...,...,...,...,...,...,...,...,...
245,"Census Tract 105.01, Providence County, Rhode ...",69185.0,2246.0,44,007,010501,2020,RI,44007010501
246,"Census Tract 105.02, Providence County, Rhode ...",70612.0,1904.0,44,007,010502,2020,RI,44007010502
247,"Census Tract 106, Providence County, Rhode Island",58203.0,2758.0,44,007,010600,2020,RI,44007010600
248,"Census Tract 107.01, Providence County, Rhode ...",91938.0,968.0,44,007,010701,2020,RI,44007010701


In [46]:
weighted_df.isna().sum()

B19013_001E     54
B01001A_017E     0
dtype: int64

In [51]:
df_2020

,,,,,,,,B19013_001E-2020,B01001A_017E-2020
GEOID,state_fips,state_name,state_usps,county_fips,county_name,tract_fips,tract_dec,,
10001040100,10,Delaware,DE,001,Kent,040100,401,64381.0,2991.0
10001040201,10,Delaware,DE,001,Kent,040201,402.01,66170.0,1443.0
10001040202,10,Delaware,DE,001,Kent,040202,402.02,NaN,NaN
10001040203,10,Delaware,DE,001,Kent,040203,402.03,56920.0,1574.0
10001040204,10,Delaware,DE,001,Kent,040204,402.04,86289.0,1956.0
...,...,...,...,...,...,...,...,...,...
44009051502,44,Rhode Island,RI,009,Washington,051502,515.02,84129.0,3843.0
44009051503,44,Rhode Island,RI,009,Washington,051503,515.03,69853.0,1969.0
44009051504,44,Rhode Island,RI,009,Washington,051504,515.04,45865.0,2373.0


In [53]:
df_2020_dups = df_2020[(df_2020.duplicated()) & (df_2020.isna().sum(axis=1) < 2)]
df_2020_drop_dups = df_2020[~df_2020.index.isin(df_2020_dups.index)]

df_2020_drop_dups.d

,,,,,,,,B19013_001E-2020,B01001A_017E-2020
GEOID,state_fips,state_name,state_usps,county_fips,county_name,tract_fips,tract_dec,,
10001040100,10,Delaware,DE,001,Kent,040100,401,64381.0,2991.0
10001040201,10,Delaware,DE,001,Kent,040201,402.01,66170.0,1443.0
10001040202,10,Delaware,DE,001,Kent,040202,402.02,NaN,NaN
10001040203,10,Delaware,DE,001,Kent,040203,402.03,56920.0,1574.0
10001040204,10,Delaware,DE,001,Kent,040204,402.04,86289.0,1956.0
...,...,...,...,...,...,...,...,...,...
44009051306,44,Rhode Island,RI,009,Washington,051306,513.06,120176.0,3028.0
44009051400,44,Rhode Island,RI,009,Washington,051400,514,81250.0,2442.0
44009051502,44,Rhode Island,RI,009,Washington,051502,515.02,84129.0,3843.0


In [9]:
census_data._collapse_df(df)



2023-12-14 17:23:18,763 - logger - INFO - B19013_001E
2023-12-14 17:23:19,101 - logger - INFO - B01001A_017E
2023-12-14 17:23:19,505 - logger - INFO - Collapsed pre-2020 data: 
B19013_001E, B01001A_017E
"{'2010': 63292.0,  '2011': 70055.0,  '2012': 74211.0,  '2013': 71141.0,  '2014': 61270.0,  '2015': 59094.0,  '2016': 57375.0,  '2017': 63324.0,  '2018': 63826.0,  '2019': 75194.0}", "{'2010': 2946.0,  '2011': 2872.0,  '2012': 2820.0,  '2013': 2897.0,  '2014': 3062.0,  '2015': 3060.0,  '2016': 3179.0,  '2017': 3378.0,  '2018': 3163.0,  '2019': 3211.0}"
"{'2010': 52369.63,  '2011': 54774.86,  '2012': 49298.17,  '2013': 52861.5,  '2014': 51109.4,  '2015': 51433.09,  '2016': 50749.6,  '2017': 50214.95,  '2018': 52084.99,  '2019': 54824.71}", "{'2010': 1922.92,  '2011': 2053.81,  '2012': 1916.94,  '2013': 1793.06,  '2014': 1818.06,  '2015': 1857.12,  '2016': 1725.28,  '2017': 1860.16,  '2018': 1731.39,  '2019': 1679.49}"
"{'2010': 62210.0,  '2011': 67655.0,  '2012': 74167.0,  '2013': 66998.

,B19013_001E,B01001A_017E
GEOID_TRACT_20,,
10001040100,"{'2010': 63292.0, '2011': 70055.0, '2012': 742...","{'2010': 2946.0, '2011': 2872.0, '2012': 2820...."
10001040201,"{'2010': 52369.63, '2011': 54774.86, '2012': 4...","{'2010': 1922.92, '2011': 2053.81, '2012': 191..."
10001040204,"{'2010': 62210.0, '2011': 67655.0, '2012': 741...","{'2010': 4360.0, '2011': 4456.0, '2012': 4946...."
10003016901,"{'2010': 55000.0, '2011': 59630.0, '2012': 594...","{'2010': 837.0, '2011': 860.0, '2012': 859.0, ..."
10001040205,"{'2010': 62210.0, '2011': 67655.0, '2012': 741...","{'2010': 4360.0, '2011': 4456.0, '2012': 4946...."
...,...,...
42101980905,NaN,"{'2010': 0.0, '2011': 0.0, '2012': 0.0, '2013'..."
42101980906,NaN,"{'2010': 0.0, '2011': 0.0, '2012': 0.0, '2013'..."
42119980101,NaN,"{'2010': 3.0, '2011': 15.0, '2012': 16.0, '201..."
